In [1]:
print('from\nhttps://aws.amazon.com/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/')

from
https://aws.amazon.com/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/


In [2]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-west-2 region. You will use the 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [3]:
bucket_name = 'jjzwest2-ml-sagemaker' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully in region ' + my_region)
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
import urllib
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [5]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


# Step 4 Train

In [15]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
print('successfully uploaded file to s3: train.csv')
s3_data='s3://{}/{}/train'.format(bucket_name, prefix)
print(f's3_data={s3_data}')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data=s3_data, content_type='csv')
print(f's3_input_train={s3_input_train}')

successfully uploaded file to s3: train.csv
s3_data=s3://jjzwest2-ml-sagemaker/sagemaker/DEMO-xgboost-dm/train
s3_input_train=<sagemaker.inputs.TrainingInput object at 0x7f11c417fa58>


In [17]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[my_region],role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

### 4c. After a few minutes, you should see training logs.

In [18]:
xgb.fit({'train': s3_input_train})

2020-12-27 00:51:40 Starting - Starting the training job...
2020-12-27 00:52:04 Starting - Launching requested ML instancesProfilerReport-1609030300: InProgress
......
2020-12-27 00:53:05 Starting - Preparing the instances for training......
2020-12-27 00:54:06 Downloading - Downloading input data...
2020-12-27 00:54:38 Training - Training image download completed. Training in progress.
2020-12-27 00:54:38 Uploading - Uploading generated training modelArguments: train
[2020-12-27:00:54:33:INFO] Running standalone xgboost training.
[2020-12-27:00:54:33:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-12-27:00:54:33:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8436.89mb
[2020-12-27:00:54:33:INFO] Determined delimiter of CSV input is ','
[00:54:33] S3DistributionType set as FullyReplicated
[00:54:33] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[00:54:33] src/tr


2020-12-27 00:55:07 Completed - Training job completed
Training seconds: 48
Billable seconds: 48


# Step 5 Deploy

In [24]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

---------------!

### 5b. To predict whether customers in the test data enrolled for the bank product or not

In [26]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
print(f'test_data_array={test_data_array}')
#xgb_predictor.content_type = 'text/csv' # set the data type for an inference
#xgb_predictor.serializer = csv_serializer # set the serializer type
#print(f'xgb_predictor set')
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

test_data_array=[[ 29   2 999 ...   0   1   0]
 [ 30  10 999 ...   0   1   0]
 [ 32   1 999 ...   0   1   0]
 ...
 [ 36   4 999 ...   0   1   0]
 [ 42   1 999 ...   0   1   0]
 [ 27   1 999 ...   0   1   0]]


ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: [[ 29   2 999 ...   0   1   0]
 [ 30  10 999 ...   0   1   0]
 [ 32   1 999 ...   0   1   0]
 ...
 [ 36   4 999 ...   0   1   0]
 [ 42   1 999 ...   0   1   0]
 [ 27   1 999 ...   0   1   0]], type: <class 'numpy.ndarray'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object

# Step 6 Evaluate

In [ ]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

# Step 7 Terminate

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()